In [1]:
import pandas as pd
from utils import connection as con
from etl import extract as ex
from etl import load
from etl import transform as t

In [2]:
conn_db = con.connect_db()
conn_dw = con.connect_dw()

In [3]:
# Pasamos los datos de las tablas a su DataFrame

df_categoria = ex.extract_only_table("Categoria", conn_db)
df_subcategoria = ex.extract_only_table("SubCategoria", conn_db)
df_carta = ex.extract_only_table("Carta", conn_db)

df_mesa = ex.extract_only_table("Mesa", conn_db)
df_clientes = ex.extract_only_table("Clientes", conn_db)
df_distrito = ex.extract_only_table("Distrito", conn_db).rename(columns={'Descripcion' : 'Distrito'})
df_direccion = ex.extract_only_table("Direcciones", conn_db)
df_local = ex.extract_only_table("Local", conn_db)
df_trabajador = ex.extract_only_table("Trabajador", conn_db)

df_pedidosdetalle = ex.extract_only_table("PedidosDetalle", conn_db)


Reemplazando IDs por descripciones

In [4]:
# Reemplazamos el idDistrito por su descripción.
df_direccion_dw = df_direccion.merge(df_distrito, on='idDistrito', how='left')[['idDirecciones','Descripcion','Distrito','idClientes','Estado']]

df_local_dw = df_local.merge(df_distrito, on='idDistrito', how='left')[['idLocal','NombreLocal','Direccion','Distrito','HorarioAtencion']]


In [5]:
# Reemplazamos el idCategoria y idSubCategoria para la carta y ponemos sus descripciones.
df_carta_dw = df_carta.merge(df_subcategoria.rename(columns={'nombre' : 'SubCategoria'}), on='idSubCategoria', how='left')\
                      .merge(df_categoria.rename(columns={'Nombre' : 'Categoria'}), on='idCategoria', how='left')\
                      [['idCarta','Nombre','Descripcion','Precio','Estado','Categoria','SubCategoria']]


In [6]:
# Reemplazamos el idLocal por su descripción.
df_trabajador_dw = df_trabajador.merge(df_local.rename(columns={'NombreLocal' : 'Local'}), on='idLocal', how='left')\
                                [['idTrabajador','ApellidosNombres','Local']]


Cargando los datos a las dimensiones

In [ ]:
load.load_dw("Dim_Carta", conn_dw, df_carta_dw)

In [9]:
load.load_dw("Dim_Mesa", conn_dw, df_mesa)

In [10]:
load.load_dw("Dim_Local", conn_dw, df_local_dw)

In [11]:
load.load_dw("Dim_Trabajador", conn_dw, df_trabajador_dw)

In [12]:
load.load_dw("Dim_Clientes", conn_dw, df_clientes)

In [13]:
load.load_dw("Dim_Direcciones", conn_dw, df_direccion_dw)

In [8]:
queryT = """
SELECT DISTINCT 
DATE_FORMAT(Fecha, '%%Y%%m%%d') AS idTiempo,
YEAR(Fecha) AS Anio,
MONTH(Fecha) AS Mes,
DAY(Fecha) AS Dia,
QUARTER(Fecha) AS Trimestre,
DAYOFWEEK(Fecha) AS DiaSemana,
DAYOFYEAR(Fecha) AS DiaAnio,
WEEK(Fecha) AS SemanaAnio,
CASE WHEN DAYOFWEEK(Fecha) IN (1,7) THEN TRUE ELSE FALSE END AS FinSemana
FROM Pedidos
"""

df_dim_tiempo = t.transform_time_from_pedido(queryT, conn_db)

In [13]:
load.load_dw("Dim_Tiempo", conn_dw, df_dim_tiempo)

In [9]:
queryP = """
SELECT
P.idPedidos, P.idLocal, P.idMesa, P.idClientes, P.idTrabajador, P.Fecha, P.TipoConsumo, P.idDireccionEnvio,
SUM(D.Cantidad) AS ItemsTotal, 
SUM(D.Precio) AS PrecioTotal,
DATE_FORMAT(P.Fecha, '%%Y%%m%%d') AS idTiempo
FROM Pedidos P
JOIN PedidosDetalle D ON P.idPedidos = D.idPedidos
JOIN dw_restaurante.Dim_Tiempo T ON DATE_FORMAT(P.Fecha, '%%Y%%m%%d') = T.idTiempo
GROUP BY P.idPedidos
"""

df_pedidos = t.transform_time_from_pedido(queryP, conn_db)

In [11]:

load.load_dw("Hec_Fact_Pedidos", conn_dw, df_pedidos)

In [12]:
load.load_dw("Dim_PedidosDetalle", conn_dw, df_pedidosdetalle)

In [13]:
## Prueba
Tablas = ["Tab01", "Tab02", "Tab03"]
for x in Tablas:
  print(x)

Tab01
Tab02
Tab03
